In [1]:
!pip install prophet ipywidgets openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from prophet import Prophet
from IPython.display import display
import ipywidgets as widgets


In [3]:
MASTER_DF = None   # Stores all historical + appended data
MODEL = None       # Trained Prophet model


In [4]:
def humanize(n):
    if n >= 1e7:
        return f"{n/1e7:.2f} Cr"
    elif n >= 1e5:
        return f"{n/1e5:.2f} L"
    elif n >= 1e3:
        return f"{n/1e3:.2f} K"
    return str(int(n))


In [5]:
upload_new = widgets.FileUpload(accept='.csv,.xlsx', multiple=False)
upload_append = widgets.FileUpload(accept='.csv,.xlsx', multiple=False)

load_btn = widgets.Button(
    description="Load / Append Data",
    button_style="info"
)

display(widgets.HTML("<b>Upload Initial Dataset</b>"))
display(upload_new)

display(widgets.HTML("<b>Append New Dataset</b>"))
display(upload_append)

display(load_btn)


HTML(value='<b>Upload Initial Dataset</b>')

FileUpload(value={}, accept='.csv,.xlsx', description='Upload')

HTML(value='<b>Append New Dataset</b>')

FileUpload(value={}, accept='.csv,.xlsx', description='Upload')

Button(button_style='info', description='Load / Append Data', style=ButtonStyle())

✅ Detected date column: 'date'
✅ Detected transaction column: 'total upi transaction'
✅ Initial dataset loaded
📊 Total records: 468


,ds,y
463,2026-01-08,64021362
464,2026-01-09,65394781
465,2026-01-10,66298891
466,2026-01-11,65646147
467,2026-01-12,68469182


In [6]:
def read_file(upload):
    key = list(upload.value.keys())[0]
    content = upload.value[key]['content']

    if key.endswith('.csv'):
        df = pd.read_csv(pd.io.common.BytesIO(content))
    else:
        df = pd.read_excel(pd.io.common.BytesIO(content))

    # Normalize column names (keep original for display)
    original_cols = df.columns.tolist()
    df.columns = [c.lower().strip() for c in df.columns]

    # ---- DATE COLUMN ----
    date_candidates = ['date', 'txn_date', 'transaction_date']
    date_col = None
    for c in date_candidates:
        if c in df.columns:
            date_col = c
            break

    if date_col is None:
        raise ValueError(
            f"❌ No date column found. Available columns: {original_cols}"
        )

    df.rename(columns={date_col: 'ds'}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'])

    # ---- TRANSACTION COLUMN (AUTO: NUMERIC + NOT DATE) ----
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if len(numeric_cols) == 0:
        raise ValueError(
            f"❌ No numeric column found for transactions. Columns: {original_cols}"
        )

    # If multiple numeric columns, pick the largest-scale one (most likely txn count)
    txn_col = max(numeric_cols, key=lambda c: df[c].mean())

    df.rename(columns={txn_col: 'y'}, inplace=True)

    print(f"✅ Detected date column: '{date_col}'")
    print(f"✅ Detected transaction column: '{txn_col}'")

    return df[['ds', 'y']]


In [7]:
def load_data(b):
    global MASTER_DF

    if not upload_new.value and not upload_append.value:
        print("❌ Please upload at least one file.")
        return

    if upload_new.value:
        MASTER_DF = read_file(upload_new)
        print("✅ Initial dataset loaded")

    if upload_append.value:
        new_df = read_file(upload_append)
        MASTER_DF = (
            pd.concat([MASTER_DF, new_df])
            .drop_duplicates('ds')
            .sort_values('ds')
        )
        print("➕ New data appended")

    print(f"📊 Total records: {len(MASTER_DF)}")
    display(MASTER_DF.tail())


In [8]:
load_btn.on_click(load_data)


In [9]:
def get_india_holidays(start, end):
    dates = pd.date_range(start, end)
    holidays = []

    for d in dates:
        if d.month == 10 and d.day in [2, 24]:   # Gandhi Jayanti / Dussehra
            holidays.append(d)
        if d.month == 11 and d.day in [12, 14]:  # Diwali window
            holidays.append(d)

    return pd.DataFrame({
        'ds': holidays,
        'holiday': 'india_festival'
    })


In [10]:
def train_model():
    global MODEL

    if MASTER_DF is None or MASTER_DF.empty:
        raise ValueError("No data loaded")

    holidays = get_india_holidays(
        MASTER_DF['ds'].min(),
        MASTER_DF['ds'].max()
    )

    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        holidays=holidays
    )

    # 🔥 ACTUAL TRAINING HAPPENS HERE
    m.fit(MASTER_DF)

    MODEL = m


In [11]:
horizon = widgets.IntSlider(
    value=30,
    min=7,
    max=365,
    step=7,
    description="Forecast Days"
)

run_btn = widgets.Button(
    description="Generate Forecast",
    button_style="success"
)

display(horizon, run_btn)


IntSlider(value=30, description='Forecast Days', max=365, min=7, step=7)

Button(button_style='success', description='Generate Forecast', style=ButtonStyle())

In [12]:
out = widgets.Output()
display(out)


Output()

In [13]:
def calculate_mape(actual, predicted):
    actual = np.array(actual)
    predicted = np.array(predicted)
    return np.mean(np.abs((actual - predicted) / actual)) * 100


In [14]:
def on_run_clicked(b):
    with out:
        out.clear_output()

        if MASTER_DF is None or MASTER_DF.empty:
            print("❌ Please load data using the Load / Append button.")
            return

        # ---------------- TRAIN MODEL ----------------
        train_model()

        # ---------------- FORECAST ----------------
        future = MODEL.make_future_dataframe(periods=horizon.value)
        forecast = MODEL.predict(future)

        df_plot = forecast[['ds', 'yhat']].tail(horizon.value)

        # ---------------- MAX PROJECTED VALUE ----------------
        max_val = df_plot['yhat'].max()
        max_date = df_plot.loc[df_plot['yhat'].idxmax(), 'ds']

        # ---------------- GROWTH % ----------------
        last_actual = MASTER_DF['y'].iloc[-1]
        last_forecast = df_plot['yhat'].iloc[-1]
        growth_pct = ((last_forecast - last_actual) / last_actual) * 100

        # ---------------- ERROR % (MAPE) ----------------
        TEST_DAYS = min(30, len(MASTER_DF) // 3)

        train_df = MASTER_DF.iloc[:-TEST_DAYS]
        test_df = MASTER_DF.iloc[-TEST_DAYS:]

        temp_model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False
        )
        temp_model.fit(train_df)

        future_test = temp_model.make_future_dataframe(periods=TEST_DAYS)
        forecast_test = temp_model.predict(future_test).tail(TEST_DAYS)

        error_pct = calculate_mape(test_df['y'], forecast_test['yhat'])

        # ---------------- PLOT ----------------
        plt.figure(figsize=(12, 6))
        plt.plot(MASTER_DF['ds'], MASTER_DF['y'], label='Actual')
        plt.plot(df_plot['ds'], df_plot['yhat'], label='Forecast')
        plt.scatter(max_date, max_val)

        plt.annotate(
            f"Max: {humanize(max_val)}",
            (max_date, max_val),
            xytext=(0, 10),
            textcoords="offset points",
            ha='center'
        )

        plt.xlabel("Date")
        plt.ylabel("Transaction Count")
        plt.legend()
        plt.show()

        # ---------------- TEXT SUMMARY BELOW GRAPH ----------------
        print("📌 Forecast Summary")
        print(f"• Maximum projected transaction count: {humanize(max_val)} on {max_date.date()}")
        print(f"• Expected growth over selected period: {growth_pct:.2f}%")
        print(f"• Model error (MAPE, last {TEST_DAYS} days): {error_pct:.2f}%")

        # ---------------- TABLE ----------------
        display(
            df_plot.assign(
                Forecast=lambda x: x['yhat'].apply(humanize)
            )
        )


In [15]:
run_btn.on_click(on_run_clicked)
